In [11]:
import numpy as np
from thirdai import bolt

In [4]:
from utils import (
    gen_single_sparse_layer_network,
    gen_numpy_training_data,
    get_categorical_acc,train_network,build_simple_distributed_bolt_network,
    train_single_node_distributed_network
)

In [5]:
def build_sparse_hidden_layer_classifier(input_dim, sparse_dim, output_dim, sparsity):
    layers = [
        bolt.FullyConnected(
            dim=sparse_dim,
            sparsity=sparsity,
            activation_function="ReLU",
        ),
        bolt.FullyConnected(dim=output_dim, activation_function="Softmax"),
    ]
    network = bolt.DistributedNetwork(layers=layers, input_dim=input_dim)
    return network

In [6]:
network = build_simple_distributed_bolt_network(
    sparsity=1.0,n_classes=10
)

Initializing Bolt network...
========= Bolt Network =========
InputLayer (Layer 0): dim=10
FullyConnectedLayer (Layer 1): dim=50, sparsity=1, act_func=ReLU
FullyConnectedLayer (Layer 2): dim=10, sparsity=1, act_func=Softmax
Initialized Network in 0 seconds


In [7]:
examples, labels = gen_numpy_training_data(n_classes=10, n_samples=1000)
train_single_node_distributed_network(network, examples, labels, epochs=1,update_parameters=False)

Distributed Network initialization done on this Node


In [8]:
x=network.get_indexed_sketch_for_gradients(layer_index=0,compression_density=0.1,sketch_biases=False,seed_for_hashing=0)
print(x)

(array([  0, 236, 423,   8, 226,  91,  25,  45, 122,  47,   6,  61,  28,
         0, 331, 342, 321,  48,  22, 216, 213,   9, 422, 427,   4,  49,
       220, 233,  59, 195, 469, 345, 435, 194, 416, 394, 379, 399, 319,
        41, 202, 425, 317, 436,  57,   0, 388, 439, 432,  62],
      dtype=uint64), array([ 0.        ,  0.01412308, -0.02415516,  0.02371759, -0.02431246,
       -0.01096685, -0.01025749,  0.02444051, -0.01479826, -0.01060372,
        0.01152415,  0.02190923,  0.01189278,  0.        , -0.01715592,
       -0.02053524, -0.0131745 ,  0.01553754,  0.01280503, -0.02117209,
       -0.01701788, -0.01590035,  0.02449853, -0.01338756,  0.01383893,
       -0.02605157, -0.01663529, -0.0143596 , -0.01675513,  0.01302834,
        0.02503607, -0.03755009, -0.02333   , -0.01167312, -0.01193696,
        0.01275214, -0.01091116, -0.01021068, -0.01518909,  0.01116666,
       -0.0113823 ,  0.01393337, -0.01286075,  0.02223918,  0.02110592,
        0.        ,  0.01489704,  0.01396014, -0.02

In [9]:
network.set_gradients_from_indices_values(layer_index=0,indices=x[0],values=x[1],set_biases=False)

In [12]:
network_weights=network.get_weights_gradients(0).flatten()
np.add.at(network_weights,x[0],-1*x[1])
y=np.linalg.norm(network_weights)

In [13]:
print(y)

0.0


In [14]:
print(np.linalg.norm(network.get_weights_gradients(0).flatten()))

0.122288436


In [10]:
zz=np.array([[1,2,3],[4,5,6]])

In [11]:
zz.flatten()

array([1, 2, 3, 4, 5, 6])

In [13]:
weights=network.get_weights_gradients(0)
print(weights)

[[-0.01304865  0.          0.00749627  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.00597566
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.02262078  0.          0.03529933  0.
   0.          0.          0.          0.02103245]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-0.02571496  0.01233628  0.01483667  0.          0.01161456  0.
   0.          0.         -0.02421263 -0.01477715]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -0.01141262  0.          0.          0.          0.
  -0.01705274  0.          0.          0.        ]
 [ 0.          0.         -0.01268313  0.          0.   

In [14]:
network.set_gradients_from_indices_values(layer_index=0,indices=x[0],values=x[1],set_biases=False)

In [15]:
weights_new=network.get_weights_gradients(0)
print(weights_new)

[[-0.01304865  0.          0.00749627  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.00597566
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.02262078  0.          0.03529933  0.
   0.          0.          0.          0.02103245]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-0.02571496  0.01233628  0.01483667  0.          0.01161456  0.
   0.          0.         -0.02421263 -0.01477715]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -0.01141262  0.          0.          0.          0.
  -0.01705274  0.          0.          0.        ]
 [ 0.          0.         -0.01268313  0.          0.   